In [1]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import requests
from selectolax.parser import HTMLParser

cdp_no = 44120
resp = requests.get(f'https://mkr.tools/api/v1/cdp/{cdp_no}').json()[0]
resp

{'act': 'lock',
 'art': 2300.121665072976,
 'block': 8894818,
 'deleted': False,
 'id': 44120,
 'ink': 29.925697539219303,
 'ire': 2150.099444469859,
 'lad': '0x3E103F903C035DA48706d5B2F956676188Ac34eF',
 'pip': 183.5875,
 'pep': 669.59875575595,
 'per': '1.0463396950513022',
 'ratio': 249.92475951640117,
 'liq_price': 110.18566168988482,
 'tab': 5748.5735400182775,
 'time': '2019-11-08T07:11:30.000Z',
 'timestamp': 1573197090000,
 'arg': None,
 'eth': None}

In [2]:
# https://developer.makerdao.com/dai/1/graphql/types
'''
type Cup {
  act: Act!            # Most recent cup action
  art: BigFloat!       # Outstanding debt DAI
  block: Int!          # Block at most recent action
  deleted: Boolean!    # True if the cup has been shut
  id: Int!             # Unique Cup Id
  ink: BigFloat!       # Collateral PETH
  ire: BigFloat!       # Collateral less fee
  lad: String!         # Cup owner
  pip: BigFloat!       # Current USD/ETH price
  ratio: BigFloat      # Current collateralisation ratio
  tab: BigFloat        # Collateral USD
  time: Datetime!      # Timestamp of most recent action
  actions: [CupAct!]   # Cup actions
}
'''

'\ntype Cup {\n  act: Act!            # Most recent cup action\n  art: BigFloat!       # Outstanding debt DAI\n  block: Int!          # Block at most recent action\n  deleted: Boolean!    # True if the cup has been shut\n  id: Int!             # Unique Cup Id\n  ink: BigFloat!       # Collateral PETH\n  ire: BigFloat!       # Collateral less fee\n  lad: String!         # Cup owner\n  pip: BigFloat!       # Current USD/ETH price\n  ratio: BigFloat      # Current collateralisation ratio\n  tab: BigFloat        # Collateral USD\n  time: Datetime!      # Timestamp of most recent action\n  actions: [CupAct!]   # Cup actions\n}\n'

CDP (44120) is **SAFE** emoji

**Debt**: 2,300.12 DAI

**Available (Max)**: 1,685.9 DAI

**Collateral**: 31.01 ETH

**Free (Max)**: 13.11 ETH

**ETH Price**: $192.81

**Liquidation**: $111.26

**Ratio**: 38.46%

In [3]:
import emoji

msg = f"""
CDP ({resp['id']}) is {'**SAFE** 😁' if round(100*resp['art'] / resp['tab'], 2) < 66.5 else '**RISKY** 😨'}
**Collateral USD**: ${round(resp['tab'], 2)}
**Debt**: {round(resp['art'], 2)} DAI
**Available (Max)**: {round(resp['tab']*0.6665585 - resp['art'], 2)} DAI
**Collateral**: {round(resp['ink']*float(resp['per']), 2)} ETH
**Free (Max)**: {round(resp['ink']*float(resp['per']) - resp['art']/(0.6665585*resp['pip']), 2)} ETH
**ETH Price**: ${round(resp['pip'], 2)}
**Liquidation**: ${round(resp['liq_price'], 2)}
**Ratio**: {round(100*resp['art'] / resp['tab'], 2)}%
"""

In [4]:
print(msg)


CDP (44120) is **SAFE** 😁
**Collateral USD**: $5748.57
**Debt**: 2300.12 DAI
**Available (Max)**: 1531.64 DAI
**Collateral**: 31.31 ETH
**Free (Max)**: 12.52 ETH
**ETH Price**: $183.59
**Liquidation**: $110.19
**Ratio**: 40.01%



In [14]:
import requests

def check_cdp(cdp_no='44120'):
    resp = requests.get(f'https://mkr.tools/api/v1/cdp/{cdp_no}').json()[0]
    msg = f"""
CDP ({resp['id']}) is {'**SAFE** 😁' if round(100*resp['art'] / resp['tab'], 2) < 66.5 else '**RISKY** 😨'}
**Collateral USD**: ${round(resp['tab'], 2)}
**Debt**: {round(resp['art'], 2)} DAI
**Available (Max)**: {round(resp['tab']*0.6665585 - resp['art'], 2)} DAI
**Collateral**: {round(resp['ink']*float(resp['per']), 2)} ETH
**Free (Max)**: {round(resp['ink']*float(resp['per']) - resp['art']/(0.6665585*resp['pip']), 2)} ETH
**ETH Price**: ${round(resp['pip'], 2)}
**Liquidation**: ${round(resp['liq_price'], 2)}
**Ratio**: {round(100*resp['art'] / resp['tab'], 2)}%
"""
    print(msg)

check_cdp()


CDP (44120) is **SAFE** 😁
**Collateral USD**: $5885.3
**Debt**: 2300.12 DAI
**Available (Max)**: 1622.77 DAI
**Collateral**: 31.02 ETH
**Free (Max)**: 12.83 ETH
**ETH Price**: $189.75
**Liquidation**: $111.24
**Ratio**: 39.08%



In [15]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.combining import OrTrigger
from apscheduler.triggers.cron import CronTrigger
import requests

trigger = OrTrigger([
   CronTrigger(hour='7', minute='30-59'),
   CronTrigger(hour='8-22', minute='*'),
   CronTrigger(hour='23', minute='0-30')
])

def check_cdp(cdp_no=44120):
    try:
        resp = requests.get(f'https://mkr.tools/api/v1/cdp/{cdp_no}').json()[0]
        msg = f"""
CDP ({resp['id']}) is {'**SAFE** 😁' if round(100*resp['art'] / resp['tab'], 2) < 66.5 else '**RISKY** 😨'}
**Collateral USD**: ${round(resp['tab'], 2)}
**Debt**: {round(resp['art'], 2)} DAI
**Available (Max)**: {round(resp['tab']*0.6665585 - resp['art'], 2)} DAI
**Collateral**: {round(resp['ink']*float(resp['per']), 2)} ETH
**Free (Max)**: {round(resp['ink']*float(resp['per']) - resp['art']/(0.6665585*resp['pip']), 2)} ETH
**ETH Price**: ${round(resp['pip'], 2)}
**Liquidation**: ${round(resp['liq_price'], 2)}
**Ratio**: {round(100*resp['art'] / resp['tab'], 2)}%
"""
    except requests.exceptions.RequestException as e:
        msg='Something went wrong! Caught requests.exceptions.RequestException.'
    print(msg)

sched = BackgroundScheduler()
sched.add_job(check_cdp, trigger, [44120])
sched.start()